In [1]:
from scipy.spatial import cKDTree
import numpy as np
import pickle
import time
from PIL import Image
from scipy.spatial import ConvexHull

In [2]:
def getEigenN(matrix):
    cov = np.cov(matrix.T)
    val, vec = np.linalg.eig(cov)
    srt = val.argsort()[::-1]
    val = val[srt]
    vec = vec[:,srt]
    if (val[2] < 0):
        val[2] = 0
    val = val/sum(val)
    return val, vec

In [3]:
def getHighIntensityRoadPoints(data, threshold, clf):
    XYZ = data[:,0:3]
    print("building KDTree")
    XYZKD = cKDTree(XYZ, leafsize=30)
    features = []
    roadIndex = []
    

    p1 = data.shape[0]/100
    p2 = 0
    #t1 = time.time()
    #t2 = time.time()
    print("go")
    
    for i in range(data.shape[0]):
        #intensity index
        if (data[i][3] >= 190):
            neighborIndex = XYZKD.query_ball_point(XYZ[i], 1)
            if len(neighborIndex) >= 3:
                neighbors = XYZ[neighborIndex]
                val, vec = getEigenN(neighbors)
                vec = vec.T
                EV1, EV2, EV3 = val[0],val[1],val[2]
                VT1, VT2, VT3 = vec[0],vec[1],vec[2]
        
                for j in range(3):
                    VT3[j] = abs(VT3[j])
                vertical = -(VT3[2]/sum(VT3) - 1)
                
                feature = [(EV1-EV2)/EV1, (EV2-EV3)/EV1, EV3/EV1, vertical]
                features.append(feature)
                roadIndex.append(i)
        
        if (i > p1):
            p1 += data.shape[0]/100
            p2 += 1
            print(str(p2)+"%")
        
        
    results = clf.predict(features)
    XYZIIndexes = []
    for i in range(len(results)):
        if (results[i] == 1):
            XYZIIndexes.append(roadIndex[i])
    return data[XYZIIndexes]

In [51]:
class RoadMarkings():
    points = None
    length = None
    width = None
    area = None
    vec = None
    val = None
    center = None
    def __init__(self, points):
        if type(points) != np.ndarray:
            self.points = np.array(points)
        else:
            self.points = points
        self.__setProperties()
    
    
    def __setProperties(self):
        rec, area = self.__minimumBoundingRectangle(self.points)
        edges = []
        for i in range(1,4):
            edges.append(np.linalg.norm(rec[0]-rec[i]))
        edges.sort()
        self.width = edges[0]
        self.length = edges[1]
        self.area = area
        self.val, self.vec = self.getEigenN(self.points)
        self.center = np.average(self.points, axis=0)
        
    def getEigenN(self, matrix):
        cov = np.cov(matrix.T)
        val, vec = np.linalg.eig(cov)
        srt = val.argsort()[::-1]
        val = val[srt]
        vec = vec[:,srt]

        if (val[1] < 0):
            val[1] = 0
        val = val/sum(val)
    
    
    
        return val, vec
    
    def __minimumBoundingRectangle(self, points):
        
        from scipy.ndimage.interpolation import rotate
        pi2 = np.pi/2.

        # get the convex hull for the points
        conv = ConvexHull(points)
        hull_points = points[conv.vertices]

        # calculate edge angles
        edges = np.zeros((len(hull_points)-1, 2))
        edges = hull_points[1:] - hull_points[:-1]

        angles = np.zeros((len(edges)))
        angles = np.arctan2(edges[:, 1], edges[:, 0])

        angles = np.abs(np.mod(angles, pi2))
        angles = np.unique(angles)

        # find rotation matrices
        # XXX both work
        rotations = np.vstack([
            np.cos(angles),
            np.cos(angles-pi2),
            np.cos(angles+pi2),
            np.cos(angles)]).T
        #     rotations = np.vstack([
        #         np.cos(angles),
        #         -np.sin(angles),
        #         np.sin(angles),
        #         np.cos(angles)]).T
        rotations = rotations.reshape((-1, 2, 2))

        # apply rotations to the hull
        rot_points = np.dot(rotations, hull_points.T)

        # find the bounding points
        min_x = np.nanmin(rot_points[:, 0], axis=1)
        max_x = np.nanmax(rot_points[:, 0], axis=1)
        min_y = np.nanmin(rot_points[:, 1], axis=1)
        max_y = np.nanmax(rot_points[:, 1], axis=1)

        # find the box with the best area
        areas = (max_x - min_x) * (max_y - min_y)
        best_idx = np.argmin(areas)

        # return the best box
        x1 = max_x[best_idx]
        x2 = min_x[best_idx]
        y1 = max_y[best_idx]
        y2 = min_y[best_idx]
        r = rotations[best_idx]

        rval = np.zeros((4, 2))
        rval[0] = np.dot([x1, y2], r)
        rval[1] = np.dot([x2, y2], r)
        rval[2] = np.dot([x2, y1], r)
        rval[3] = np.dot([x1, y1], r)

        return rval, conv.area

In [5]:
class Pixel():
    #size of each pixel
    size = None
    #raw input numpy array (n * m)
    rawData = None
    #translated numpy array (3 * n), minimal value = 0
    __translatedData = None
    #min, max and range of x, y and z
    shapes = None
    #range of x, y and z
    ranges = None
    #voxelized input data (numpy array xRange * yRange * zRange)
    pixels = None
    #a list of indexes of rawData
    __pointsList = None
    #cluster data
    clusters = None

    
    def __init__(self, rawData, size = 0.065, withKDTree = False):
        self.rawData = rawData
        self.size = size
        self.__getNewShapes()
        self.pixels = np.zeros(self.ranges[0:2])
        self.__pointsList = self.__makeList(self.pixels.size)
        self.__loadDataIndexToPointsList()
        self.__loadPointsListToPixels(threshold = 0)
        self.__cluster()
    def __getNewShapes(self):
        data = np.copy(self.rawData[:,0:3].T)
        maxXYZ = [np.amax(data[i]) for i in range(3)]
        minXYZ = [np.amin(data[i]) for i in range(3)]
        rangeXYZ = [int(maxXYZ[i]/self.size) - int(minXYZ[i]/self.size) + 1 for i in range(3)]
        
        self.ranges = rangeXYZ
        self.__translatedData = np.array([data[i] - minXYZ[i] for i in range(3)])
        self.shapes = {"xMax": maxXYZ[0], "yMax": maxXYZ[1], "zMax": maxXYZ[2],
                      "xMin": minXYZ[0], "yMin": minXYZ[1], "zMin": minXYZ[2], 
                      "x_range": rangeXYZ[0], "y_range": rangeXYZ[1], "z_range": rangeXYZ[2]}
    def __makeList(self, length):
        return [None] * length
    def __getPointsListIndexFromIJ(self, i, j):
        y_r = self.ranges[1]
        return i * y_r + j
    
    
    def __loadDataIndexToPointsList(self):
        scaledData = self.__translatedData / self.size
        
        #shape[1]: number of points
        for i in range(self.__translatedData.shape[1]):
            I, J = [int(scaledData[j][i]) for j in range(2)]
            #print([scaledData[j][i] for j in range(3)])
            index = self.__getPointsListIndexFromIJ(I, J)
            if not self.__pointsList[index]:
                self.__pointsList[index] = list()
            self.__pointsList[index].append(i)
            
    def __loadPointsListToPixels(self, threshold = 3):
        for vi in range(len(self.__pointsList)):
            if self.__pointsList[vi]:
                pointsInside = len(self.__pointsList[vi])
                if (pointsInside > threshold):
                    i, j = self.__getIJFromPointsListIndex(vi)
                    self.pixels[i][j] = pointsInside
    def __getIJFromPointsListIndex(self, index):
        y_r = self.ranges[1]
        i, j = divmod(index, y_r)
        return i, j
    def __cluster(self):
        #18.xxxx
        markDistance = 22
        
        
        
        rows = self.ranges[0]
        cols = self.ranges[1]

        bag = []
        kernel = [(0,-2),(0,-1),(0,1),(0,2),(-1,-1),(-1,0),(-1,1),(1,-1),(1,0),(1,1),(-2,0),(2,0)]
        intensityData = np.copy(self.pixels)
        
        #---connect parts
        for i in range(2, rows - 2):
            for j in range(2, cols - 2):

                if (intensityData[i][j] >= 1):
                    cluster = []
                    near = [(i, j)]
                    while(len(near) != 0):
                        point = near.pop()
                        cluster.append(point)
                        px, py = point
                        intensityData[px][py] = 0
                        for offset in kernel:
                            tmpx = px + offset[0]
                            tmpy = py + offset[1]
                            if (tmpx >= rows) or (tmpy >= cols):
                                continue
                            if intensityData[tmpx][tmpy] >= 1:
                                intensityData[tmpx][tmpy] = 0
                                near.append(tuple((tmpx, tmpy)))

                    bag.append(cluster)
        #connect parts---
        
        #---make mark
        self.clusters = []
        for i in range(len(bag)):
            if (len(bag[i]) > 15):
                RM = RoadMarkings(bag[i])
                self.clusters.append(RM)
        #make mark---
        
        
        #connect marks---
        dashLine = []
        crossWalk = []
        
        centers = np.array([rm.center for rm in self.clusters])
        RMKD = cKDTree(centers ,leafsize = 30)
        discovered = [False] * len(self.clusters)
        for i in range(len(discovered)):
            if not discovered[i]:
                discovered[i] = True
                if 20<self.clusters[i].length<60:
                    if 1.5<self.clusters[i].width <10:            
                        neighborIndex = RMKD.query_ball_point(centers[i], markDistance)
                        
                
            else:
                pass
        
        
        
        
        
        if 20<roadmark.length <60:
            if 1.5<roadmark.width <10:
            r = roadmark.points.tolist()
            for point in r:
                point += [0,255,255,0]
            tmp += r
            
            
        (q[1].center - q[0].center)/np.array(q[0].vec.T[1])

        #---connect marks
        

In [ ]:
def point_in_hull(point, hull, tolerance=1e-12):
    return all(
        (np.dot(eq[:-1], point) + eq[-1] <= tolerance)
        for eq in hull.equations)

In [6]:
def gray_2_rgb(img):
    
    new_img = np.zeros((img.shape[0],img.shape[1],3))
        
    for i in range(img.shape[0]):
        for k in range(img.shape[1]):
            if img[i][k] >= 1:
                new_img[i][k] = np.array([255,255,255])
    
    new_img = new_img.astype('uint8')
    
    return new_img


In [63]:
XYZI = np.genfromtxt("cross.csv", delimiter=",", usecols=(0,1,2,3))
with open('roadClassifier.pickle', 'rb') as f:
    clf = pickle.load(f)
highIntensityRoadPoints = getHighIntensityRoadPoints(XYZI, 190, clf)

building KDTree
go
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


In [14]:
a = highIntensityRoadPoints.tolist()
import csv as c
with open("QQ.csv","w") as f:
    writer = c.writer(f)
    for i in range(len(a)):
        writer.writerow(a[i])

In [64]:
b = Pixel(highIntensityRoadPoints)
q = b.clusters

In [179]:
rgb = gray_2_rgb(b.pixels)
Image.fromarray(rgb).save("QQ.png")

In [17]:
tmp = []
for roadmark in q:
    if 20<roadmark.length <55:
        if 1.5<roadmark.width <5:
            r = roadmark.points.tolist()
            for point in r:
                point += [0,255,255,0]
            tmp += r
    if 35<roadmark.length <60:
        if 5<roadmark.width <10:
            r = roadmark.points.tolist()
            for point in r:
                point += [0,110,255,135]
            tmp += r
            

In [18]:
import csv as c
with open("tr.csv","w")as f:
    wr = c.writer(f)
    for i in range(len(tmp)):
        wr.writerow(tmp[i])

In [54]:
#straight
for i in range(3):
    print(q[i].vec)

[[ 0.98427515 -0.17664212]
 [ 0.17664212  0.98427515]]
[[ 0.98302295 -0.18348265]
 [ 0.18348265  0.98302295]]
[[ 0.98112743 -0.19336229]
 [ 0.19336229  0.98112743]]


In [65]:
#cross
for i in range(3):
    print(q[i].vec)

[[ 0.15497479 -0.98791842]
 [-0.98791842 -0.15497479]]
[[ 0.16243637 -0.98671902]
 [-0.98671902 -0.16243637]]
[[ 0.14934165 -0.98878566]
 [-0.98878566 -0.14934165]]


In [66]:
for i in range(3):
    print(q[i].center)

[ 6.14563107 21.49838188]
[24.04040404 24.51515152]
[42.36567164 27.58955224]


In [69]:
(q[1].center - q[0].center)

array([17.89477297,  3.01676964])

In [88]:
(q[1].center - q[0].center)/np.array(q[0].vec.T[1])

array([-18.11361396, -19.46619595])

In [84]:
bb = np.array(q[1].vec.T[1])  

In [86]:
(aa+bb)/2

array([-0.98731872, -0.15870558])

In [172]:
for i in q:
    print(i.length, i.width, i.area)

33.375990723224334 3.015276420478211 70.10291415521363
44.71652454705713 3.287979746107138 93.01597923744686
25.317521178281183 1.9007148031742758 51.20689751716371
50.00678983855293 4.343490053952856 103.28355875253277
46.069931388591144 2.446393378947527 94.91557836774646
72.03308063579082 11.884245626780315 149.51762984400264
77.04800140527941 12.248119788610996 158.5550962860126
45.098532491630976 3.3887240004245176 93.59093442118234
44.91059494664415 3.922322702763647 92.72270969074177


In [176]:
for i in q:
    print(i.length, i.width, i.area)

12.02081528017131 2.262741699796961 25.620844288350025
41.43645737753168 6.788225099390855 89.71490618511083
41.733114091042154 6.675320381861003 91.06280794520833
42.42044954714167 7.741534554226974 95.93793000342353
43.563493680504735 7.539011689054904 95.67503880338154
95.32309525901243 80.53820156731692 308.0702968289734
9.535141263710718 3.123580758801788 22.782359378457354
54.98838030316544 9.861537006295611 120.62917485227376
12.343095235798838 2.8621670111997335 27.52200008821102
55.32491106561633 9.170768139060415 119.3418803884978
43.565731635919676 7.726752403351784 98.79456605317668
9.999999999999995 3.2000000000000206 22.53663105724556
9.214251951427753 2.3426064283290935 20.87074357711168
14.263994477758994 3.100868364730199 31.892879175670384
42.969284532127524 7.565363099442459 97.26017459120177
42.92497747553476 7.2410555795436435 95.9060195858735
42.578281925685054 6.725141108986478 94.08054764557171
10.733126291999003 4.47213595499958 24.66562126173752
